1) The environment in the Nim learning model is built for two players. Each player removes items from three piles on each of their turns. For each turn a player can remove at least one item from only one of the three piles. Whichever player clears the last pile, wins. The agent (player / robot) works in this environment to win by receiving rewards and penalties to form new states that come up with future successful moves.

2) An agent is continuopusly learning and planning by receiving rewqards and penalties for their actions. The agents strategy will show the strength by the speed of how quickly it learns to be successful or gain. 
    
   One agent in the Nim learning model is the Q-learner. The Bellman equation is used to find the expected value of states and stores their actions in the Q-table. The Q-table essentially tells the agenet the expected reward for taking action (a) in state (s). The Q values are updated throught the process to fidn the action that maximizes the expected reward for the next state (s).
    
   Another agent in the Nim learning model is the random player. The random player works against the Q-learner, and finds the non-empty piles. Then once the non-empty piles are found the random player selects one of the non-empty piles and the numberof items to remove at random.
   
   Another agent is the Nim Guru who moves with the bitwise operator mathemtical solution to the Nim game. By X-oring the item counts in piles and choosing the perfect amount of items to remove from the pile selected, the guru solves the Nim game.

3) The result of the Nim learning model if the game continues the action is a 0, therefore no reward or penalty. If the game is won by the player/agent the action/state that results in the win is a reward of 1. Lastly, if a player/agent's action/state loses its penalty is -1. 

4.How many possible states there could be in the Nim game with a maximum of 10
items per pile and 3 piles total?

In [1]:
import numpy as np
from random import randint, choice

ITEMS_MX = 10

def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  
    return randint(1, _st[pile]), pile

def nim_qlearner(_st):
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st) 

    return move, pile 

Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        state[pile] -= move
        if state == [0, 0, 0]: 
            return side  

        side = 'B' if side == 'A' else 'A' 

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    print(f"{_n} games, {a:>8s}{wins['A']:5d}  {b:>8s}{wins['B']:5d}")

    return wins['A'], wins['B']

In [2]:
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru')

1000 games,   Random  517    Random  483
1000 games,     Guru  998    Random    2
1000 games,   Random    9      Guru  991
1000 games,     Guru  940      Guru   60


(940, 60)

In [3]:
qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

def nim_qlearn(_n):
    global qtable
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    for i in range(_n):
        st1 = init_game()
        while True: 
            move, pile = nim_random(st1)
            st2 = list(st1)
            st2[pile] -= move  
            if st2 == [0, 0, 0]: 
                qtable_update(Reward, st1, move, pile, 0) 
                break  

            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)
    
nim_qlearn(1000)

In [4]:
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner')
play_games(1000, 'Random', 'Random')

1000 games, Qlearner  718    Random  282
1000 games,   Random  313  Qlearner  687
1000 games,   Random  538    Random  462


(538, 462)

In [5]:
%%time

n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Random')
    wins += [a/(a+b)]

1000 games, Qlearner  542    Random  458
1000 games, Qlearner  477    Random  523
1000 games, Qlearner  699    Random  301
1000 games, Qlearner  696    Random  304
1000 games, Qlearner  693    Random  307
1000 games, Qlearner  715    Random  285
1000 games, Qlearner  720    Random  280
CPU times: user 8.58 s, sys: 77 ms, total: 8.66 s
Wall time: 8.7 s


In [6]:
import pandas as pd

def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)

        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r

            print(s, file=fout)

qtable_log('qtable_debug.txt')

qtable_log_df = pd.read_csv('qtable_debug.txt')

print(len(qtable_log_df.index))

1331


4) There are 1331 states in the above Nim game with 10 items per pile and 3 piles total.

In [7]:
print(len(qtable_log_df.columns)-1)

30


5) There are 30 unique actions for player 1 in the above Nim game with 10 items per pile and 3 piles total.

6) Find a way to improve the provided Nim game learning model. Do you think one
can beat the Guru player?

Testing the Guru versus the Q-Learner initially to see performance before improving the model.

In [8]:
%%time

n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Guru')
    wins += [a/(a+b)]

1000 games, Qlearner    4      Guru  996
1000 games, Qlearner    6      Guru  994
1000 games, Qlearner    5      Guru  995
1000 games, Qlearner   19      Guru  981
1000 games, Qlearner   22      Guru  978
1000 games, Qlearner   32      Guru  968
1000 games, Qlearner   22      Guru  978
CPU times: user 7.69 s, sys: 49.1 ms, total: 7.74 s
Wall time: 7.73 s


In [9]:
# IMPROVING the learning model

qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0

def nim_qlearn(_n):
    global qtable
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    for i in range(_n):
        st1 = init_game()
        while True: 
            move, pile = nim_random(st1)
            st2 = list(st1)
            st2[pile] -= move  
            if st2 == [0, 0, 0]:  
                qtable_update(Reward, st1, move, pile, 0)  #Win
                break                

            #How do we implement penalties or change the reward / params
            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2
        
# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)
    
nim_qlearn(1000)

In [10]:
%%time

# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Guru')
    wins += [a/(a+b)]

1000 games, Qlearner    3      Guru  997
1000 games, Qlearner    5      Guru  995
1000 games, Qlearner   16      Guru  984
1000 games, Qlearner   16      Guru  984
1000 games, Qlearner   27      Guru  973
1000 games, Qlearner   40      Guru  960
1000 games, Qlearner   22      Guru  978
CPU times: user 7.34 s, sys: 30.2 ms, total: 7.37 s
Wall time: 7.36 s


In [11]:
%%time
alp = {0.1,0.5,0.99}
ga = {0.1,0.5,0.99}
rew = {0.1,1,10,50,100}

for al in alp:
    for g in ga:
        for r in rew:    
            Alpha = al
            Gamma = g
            Reward = r

            print("alpha = " + str(Alpha)+ " , gamma = "+ str(Gamma) + " , reward = " + str(Reward))
            n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
            wins = []
            for n in n_train:
                nim_qlearn(n)
                a, b = play_games(1000, 'Qlearner', 'Guru')
                wins += [a/(a+b)]
    

alpha = 0.1 , gamma = 0.1 , reward = 0.1
1000 games, Qlearner    2      Guru  998
1000 games, Qlearner    6      Guru  994
1000 games, Qlearner    3      Guru  997
1000 games, Qlearner   14      Guru  986
1000 games, Qlearner   34      Guru  966
1000 games, Qlearner   25      Guru  975
1000 games, Qlearner   44      Guru  956
alpha = 0.1 , gamma = 0.1 , reward = 1
1000 games, Qlearner    2      Guru  998
1000 games, Qlearner    4      Guru  996
1000 games, Qlearner    7      Guru  993
1000 games, Qlearner   14      Guru  986
1000 games, Qlearner   27      Guru  973
1000 games, Qlearner   30      Guru  970
1000 games, Qlearner   20      Guru  980
alpha = 0.1 , gamma = 0.1 , reward = 100
1000 games, Qlearner    7      Guru  993
1000 games, Qlearner    7      Guru  993
1000 games, Qlearner    3      Guru  997
1000 games, Qlearner   15      Guru  985
1000 games, Qlearner   30      Guru  970
1000 games, Qlearner   30      Guru  970
1000 games, Qlearner   25      Guru  975
alpha = 0.1 , gamm

1000 games, Qlearner   25      Guru  975
1000 games, Qlearner   31      Guru  969
1000 games, Qlearner   27      Guru  973
alpha = 0.5 , gamma = 0.99 , reward = 1
1000 games, Qlearner    4      Guru  996
1000 games, Qlearner    2      Guru  998
1000 games, Qlearner    7      Guru  993
1000 games, Qlearner   12      Guru  988
1000 games, Qlearner   30      Guru  970
1000 games, Qlearner   29      Guru  971
1000 games, Qlearner   32      Guru  968
alpha = 0.5 , gamma = 0.99 , reward = 100
1000 games, Qlearner    6      Guru  994
1000 games, Qlearner    1      Guru  999
1000 games, Qlearner    2      Guru  998
1000 games, Qlearner   13      Guru  987
1000 games, Qlearner   28      Guru  972
1000 games, Qlearner   32      Guru  968
1000 games, Qlearner   39      Guru  961
alpha = 0.5 , gamma = 0.99 , reward = 10
1000 games, Qlearner    3      Guru  997
1000 games, Qlearner    6      Guru  994
1000 games, Qlearner    8      Guru  992
1000 games, Qlearner   15      Guru  985
1000 games, Qlea

In [12]:
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)

        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r

            print(s, file=fout)

qtable_log('qtable_debug.txt')

qtable_log_df = pd.read_csv('qtable_debug.txt')

print(len(qtable_log_df.index))

1331


6) I'm having a very hard time implementing a penalty for losing. I've tried to iterate through alpha, gamma, and reward to see the best tuned parameters. I've found the best results to come from alpha = 0.99 , gamma = 0.5 , reward = 100.